In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-goa63s07/kobert-tokenizer_299f5032eccc4c27b5a836745db13827
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-goa63s07/kobert-tokenizer_299f5032eccc4c27b5a836745db13827
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook,notebook


In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:


device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
ori_t=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/kobert/감성대화말뭉치(최종데이터)_Training.xlsx')

ori_v=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/kobert/감성대화말뭉치(최종데이터)_Validation.xlsx')


In [ ]:
ori_t=ori_t.fillna("")
ori_v=ori_v.fillna("")


In [ ]:

ori_t["사람문장"]=ori_t["사람문장1"].astype(str)+ori_t["사람문장2"].astype(str)+ori_t["사람문장3"].astype(str)
ori_v["사람문장"]=ori_v["사람문장1"].astype(str)+ori_v["사람문장2"].astype(str)+ori_v["사람문장3"].astype(str)


In [ ]:
df_concat=pd.concat([ori_t,ori_v])

In [ ]:
final_df=df_concat[["사람문장","감정_대분류"]]

In [ ]:
final_df=final_df.rename({"감정_대분류":"감정"},axis=1)

In [ ]:
#컬럼 내 데이터들의 공백(string) 제거

final_df["감정"]=final_df["감정"].apply(lambda x:x.strip())

In [ ]:
final_df["감정"].value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정, dtype: int64

In [ ]:
#np.where(조건,True일때 값,False일때 값)

final_df["감정"]=np.where(final_df["감정"].str.match("상처"),"슬픔",final_df["감정"])
final_df["감정"]=np.where(final_df["감정"].str.match("기쁨"),"행복",final_df["감정"])
final_df=final_df.reset_index(drop=True)

In [ ]:

final_df.rename({"사람문장":"발화"},axis=1,inplace=True)

In [ ]:
final_df.head()

,발화,감정
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,분노


In [ ]:
kk = final_df['감정'].unique()
print(kk)

['분노' '행복' '불안' '당황' '슬픔']


In [ ]:
final_df.loc[(final_df['감정'] == "불안"), '감정'] = 0  #불안 => 0
final_df.loc[(final_df['감정'] == "당황"), '감정'] = 1  #당황 => 1
final_df.loc[(final_df['감정'] == "분노"), '감정'] = 2  #분노 => 2
final_df.loc[(final_df['감정'] == "슬픔"), '감정'] = 3  #슬픔 => 3

final_df.loc[(final_df['감정'] == "행복"), '감정'] = 4  #행복 => 4
# final_df.loc[(final_df['감정'] == "혐오"), '감정'] = 6  #혐오 => 6
# final_df.loc[(final_df['감정'] == "중립"), '감정'] = 5  #중립 => 5
data_list = []
for q, label in zip(final_df['발화'], final_df['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))




    data_list.append(data)


In [ ]:

print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '2']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.안 그래도 할 일 많은데 왜 자꾸 나한테 추가 업무를 주는지 모르겠다니깐.', '2']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.나도 졸업하고 바로 일하고 싶었는데.응. 나는 다 잘 될 줄 알았는데 이게 뭐야.', '0']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.간암에 걸렸다는 말을 왜 안 믿을까?친구가 믿지 않는 건 어쩔 수 없지만 조금 서운하네.', '3']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.그러니 불안해서 견딜 수가 있어야지. 아예 움직일 수 없을까 봐.아직 포기하긴 이르지. 할 수 있는 모든 방법을 다 써봐야지.', '3']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.지금은 없어. 난 왜 이렇게 되는일이 없나 몰라.', '3']
['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.맞아. 하지만 그렇다고 아무나하고 결혼할 수도 없잖아.', '1']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0)

In [ ]:

print(len(dataset_train))
print(len(dataset_test))

50695
7576


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:

#토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = tokenizer.tokenize


data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
data_train[0]

(array([   2, 1434, 2782, 7096,  517, 7779, 5808, 6844,   54,  889, 7779,
        7811,  517, 7942, 1434, 5439, 4279, 6557, 6903, 2485, 6150, 6667,
        1451,  517, 6322, 6855, 6324, 5770, 1550, 1833, 5868, 6136, 6288,
        5444, 4945,   54, 5655, 2934, 4128, 6855, 5330, 3942, 3135, 1763,
        5377, 5928,   54, 1201, 2782, 1849, 2958, 5859,  517, 5767, 6886,
        3969, 6241, 5859, 2044, 7074, 7318, 2087, 4299,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 데이터셋 객체에서 클래스 레이블 정보 확인
train_labels = data_train.labels
test_labels = data_test.labels

# 클래스 레이블의 고유한 값 확인
unique_train_labels = set(train_labels)
unique_test_labels = set(test_labels)

print("Unique train labels:", unique_train_labels)
print("Unique test labels:", unique_test_labels)

Unique train labels: {0, 1, 2, 3, 4}
Unique test labels: {0, 1, 2, 3, 4}


In [ ]:
emotion_dict = {}
for i in sorted(unique_train_labels):
    print(i)
    if(i==0):
      emotion_dict[i] = '불안'
    elif(i==1):
      emotion_dict[i] = '당황'
    elif(i==2):
      emotion_dict[i] = '분노'
    elif(i==3):
      emotion_dict[i] = '슬픔'
    elif(i==4):
      emotion_dict[i] = '행복'
print(emotion_dict)

0
1
2
3
4
{0: '불안', 1: '당황', 2: '분노', 3: '슬픔', 4: '행복'}


In [ ]:
print(train_dataloader)

In [ ]:
num_classes = len(set(train_labels))
print(num_classes)

5


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        print("@")
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
best_acc=0.0
best_loss=99999999

In [ ]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        # token_ids를 자연어로 디코딩

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc

      print('current best model saved')

  0%|          | 0/793 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6043554544448853 train acc 0.203125
epoch 1 batch id 201 loss 0.9619883298873901 train acc 0.41239116915422885
epoch 1 batch id 401 loss 0.7400469779968262 train acc 0.5373675187032418
epoch 1 batch id 601 loss 0.8694666028022766 train acc 0.5891482945091514
epoch 1 train acc 0.6164204647811204


<ipython-input-38-d01383dfad68>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/119 [00:00<?, ?it/s]

epoch 1 test acc 0.704875700280112 test loss 0.5924596190452576


NameError: ignored

In [ ]:
#토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = tokenizer.tokenize

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another,0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


#         test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            emotion = emotion_dict[np.argmax(logits)]


        print(f">> 입력하신 내용의 감정은 {emotion}입니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence.upper() == "QUIT":
        print("감정 분석을 종료합니다.")
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나 너무 화나


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용의 감정은 분노입니다.


하고싶은 말을 입력해주세요 : 나 너무 혼란스러워
>> 입력하신 내용의 감정은 불안입니다.


하고싶은 말을 입력해주세요 : 나 지금 어떻게 해야 할지 모르겠고, 지금 이 상황이 뭔지 모르겠어
>> 입력하신 내용의 감정은 불안입니다.


하고싶은 말을 입력해주세요 : 나 너무 당황스러워
>> 입력하신 내용의 감정은 당황입니다.




KeyboardInterrupt: ignored

In [2]:
!git remote add origin https://github.com/howon-ryu/kobert_test.git

In [20]:
cd /content/drive/MyDrive/Colab Notebooks/kobert

/content/drive/MyDrive/Colab Notebooks/kobert


In [22]:
!git init
!git config --global user.email "qorgh2akfl@naver.com"
!git config --global user.name "howon_ryu"

Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/kobert/.git/
